# DACON 재정정보 AI 검색 알고리즘 경진대회


##  1. 라이브러리 설치 및 라이브러리 Import
데이터 처리를 위해 필요한 라이브러리를 불러오고 기본 설정을 하는 셀입니다.

In [1]:
!apt-get install git -y
!apt-get install poppler-utils -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.25.1-1ubuntu3.13).
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.86.1-0ubuntu1.4).
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.


In [2]:
!pip install -q git+https://github.com/THU-MIG/yolov10.git
!pip install -q supervision
!pip install pdf2image

In [ ]:
!wget https://github.com/moured/YOLOv10-Document-Layout-Analysis/releases/download/doclaynet_weights/yolov10x_best.pt

In [3]:
!pip install pymupdf
!pip install natsort
!pip install pymupdf4llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 10.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 33.4 MB/s eta 0:00:00a 0:00:01


In [1]:
!pip install transformers==4.42.3
!pip install --upgrade torch torchvision torchaudio

   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   ------------------------- -------------- 6.0/9.3 MB 28.4 MB/s eta 0:00:01
   ---------------------------------------- 9.3/9.3 MB 30.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 63.0 MB/s eta 0:00:00


In [ ]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U
!pip install --upgrade bitsandbytes

In [2]:
import supervision as sv
import os
import cv2
import supervision as sv
from ultralytics import YOLO
import fitz
from natsort import natsorted
import json
import numpy as np
import re
import pymupdf4llm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pdf2image import convert_from_path
import os
from tqdm import tqdm


##  2.함수 선언

### PDF 파일을 PNG 이미지로 변환하여 저장
- PDF 크기를 유지한 채 이미지로 변환

In [3]:
def convert_pdf_to_png(pdf_path, output_dir):
    """PDF 파일을 PNG 이미지로 변환하여 저장"""
    pages = convert_from_path(pdf_path, dpi=300)  # dpi 설정으로 해상도 조절 가능
    for i, page in enumerate(pages):
        # 페이지 크기 가져오기 (포인트 단위)
        page_width = int(page.width * 0.352778)  # 포인트를 픽셀로 변환 (72dpi 기준)
        page_height = int(page.height * 0.352778)  # 포인트를 픽셀로 변환 (72dpi 기준)

        # 이미지 변환 (PDF 크기 유지)
        image = convert_from_path(
                pdf_path, 
                dpi=300, 
                first_page=i+1, 
                last_page=i+1,
                size=(page_width, page_height)  # 이미지 크기를 PDF 페이지 크기로 설정
        )[0]

        image.save(os.path.join(output_dir, f"page_{i+1}.png"), "PNG")


folder_path = './test_source' 
pdf_name = os.listdir(folder_path)
files_name = [f.replace('.pdf', '') for f in os.listdir(folder_path)]

pdf_path = "./test_source/"  # PDF 파일 경로
output_dir = "./dacon/test_png/"  # 이미지 저장 폴더 경로
for i in tqdm(range(len(pdf_name))):
        os.makedirs(output_dir+files_name[i], exist_ok=True)
        convert_pdf_to_png(pdf_path+pdf_name[i], output_dir+files_name[i])# 폴더가 없으면 생성
  


100%|██████████| 9/9 [00:37<00:00,  4.21s/it]


### Yolo모델의 부족한 Detection ablility를 보완합니다.
- 글자가 작고 틀이 없는 주석의 특징 때문에 불안정한 Detection을 주석의 특징을 파악하여 파서 텍스트의 문맥 끊김 방지
- 중복 된 Detection 객체를 처리 가능
- Detection의 좌표를 일정하게 변환하여 안정적으로 파서 가능
- 주로 공식 문서 좌측 상단에 있는 머리말을 제거하여 파서 텍스트의 문맥 끊김 방지
- 마크다운에서 

In [10]:
def get_number_from_filename(filename):
    match = re.search(r'page_(\d+)', filename)
    if match:
        return int(match.group(1))
    else:
        return 0  
    
def detect_filter(sorted_detections):
    filtered_detections = sorted_detections.copy()  
    
    
    i = 0
    while i < len(filtered_detections) - 1:
        if np.array_equal(filtered_detections[i]['xyxy'], filtered_detections[i + 1]['xyxy']):
            
            if filtered_detections[i + 1]['data']['class_name'] == 'Section-header':
                filtered_detections.pop(i)
            elif filtered_detections[i]['data']['class_name'] == 'Section-header':
                filtered_detections.pop(i + 1)
            else:
                i += 1
              
        else:
            i += 1  

    return filtered_detections
def width_plus(sorted_detections): 
                                    
    plus_detections = sorted_detections.copy()                                
    half = 595.0
    foot_count_l = 0
    foot_count_r = 0
    for plus_detection in plus_detections:
        x1,y1,x2,y2 = plus_detection['xyxy']
        if plus_detection['data']['class_name'] == 'Section-header':
            
            if half > x1: # 왼쪽
                x2 = 534.12
            else: #오른쪽
                x2 = 1133.4
            new_arr = np.array([x1, y1, x2, y2], dtype=np.float32)
            plus_detection['xyxy'] = new_arr
            
        elif plus_detection['data']['class_name'] == 'Footnote':
            if half > x1 and foot_count_l == 0: # 왼쪽
                x2 = 534.12
                y2 = 780
                foot_count_l += 1
            elif half < x1 and foot_count_r == 0: #오른쪽
                x2 = 1133.4
                y2 = 780
                foot_count_r += 1
            new_arr = np.array([x1, y1, x2, y2], dtype=np.float32)
            plus_detection['xyxy'] = new_arr
    return  plus_detections
vectorizer = TfidfVectorizer()
def detect_filter_head(sorted_detections):
    filtered_detections = sorted_detections.copy()  
    i = 0
    
    while i < len(filtered_detections) - 1:
        x1,y1,x2,y2 = filtered_detections[i]['xyxy']
        if y1 <=42.341:
            filtered_detections.pop(i)
    
        else:
            i += 1  
        
    return filtered_detections
def trans_table(markdown_tables,table_temp):
    max_similarity = 0
    table_text = ''
    for markdown_table in markdown_tables:
        replace_table = markdown_table
        cleaned_table = replace_table.replace('|', '').replace('#', '').replace('□', '').replace('-','').replace('\n','').replace(' ','').replace('*','')
        tfidf_matrix = vectorizer.fit_transform([table_temp, cleaned_table])
        similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
                    
        if max_similarity < similarity:
                        table_text = markdown_table
                        max_similarity = similarity
   
         
    return table_text
def transpose_detection(image,new_detections):
    image_height, image_width = image.shape[:2]
        
    if image_height < image_width: 
            half = image_width / 2
            left = []
            right = []
            
            for new_detection in new_detections:
                if new_detection['xyxy'][0] < half: 
                    left.append(new_detection)
                else:
                    right.append(new_detection)
            
            
            
            left_detections = sorted(left, key=lambda x: x['xyxy'][1])
            right_detections = sorted(right, key=lambda x: x['xyxy'][1])
            sum = left_detections + right_detections
            return sum
            
    else:
            return sorted(new_detections, key=lambda x: x['xyxy'][1])    

### LLM함수 선언


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "rtzr/ko-gemma-2-9b-it"

quantization_config_4bit = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=quantization_config_4bit,
    low_cpu_mem_usage=True,
)

model.eval()

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<end_of_turn>")
]

def make_answer(instruction):
    messages = [
        {"role": "user", "content": f"{instruction}"}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.9,
        top_p=0.9,
    )

    return tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)


Loading checkpoint shards: 100%|██████████| 10/10 [00:05<00:00,  1.96it/s]


## 3.파서함수 선언

- 파서함수 상세설명
1. Yolo모델을 기반으로 PDF에서 문맥에 맞게 Detection을 함 (과정) PDF -> .png -> Yolo Detection -> Detection 객체
2. PyMuPDF,pymupdf4ll를 이용하여 pdf 파일에서 text를 추출
3. 일정하지 않은 이미지의 크기를 일정하게 변경
4. 이전에 Detection한 객체의 좌표를 이용하여 중복 방지
5. 표를 처리하기 위해 pymupdf4ll의 라이브러리를 이용하여 Markdown을 저장하고, Cosine 유사도를 이용하여 처리
6. 사람 눈에는 보기 좋지만 컴퓨터에서 순서대로 배치하면 보기 어려운 플로우차트를 LLM을 활용하여 논리적으로 파서
7. 너무 짧은 파서 문서는 다음 파서 문서에 병합  
8. 함수 실행 시 답답함을 해소하기 위해 진행률을 시각화


In [11]:
def detect_and_parser(folder_path, model_path='yolov10x_best.pt'):
    
    model = YOLO(model_path)
    pdf_document = fitz.open('./test_source/' + folder_path.split('/')[-1] + '.pdf')
    
    markdown_document = pymupdf4llm.to_markdown(pdf_document, page_chunks=True)
    
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
    sorted_image_files = natsorted(image_files, key=get_number_from_filename)
      
    docs = {} 
    doc_id = 0
    temp = ''
    pre_class_name=''
    sub_class_name=''
    check_order = ''
    extracted_text = ''
    p_x1, p_y1, p_x2, p_y2 = 0, 0, 0, 0
    
    for i, image_file in enumerate(sorted_image_files): 
        markdown = markdown_document[i]
        markdown_tables = markdown['text'].split('\n\n\n')
        
        page = pdf_document[i] 
        page_width = int(page.rect.width)
        page_height = int(page.rect.height)
        
        image_path = os.path.join(folder_path, image_file) 
        image = cv2.imread(image_path) 
        image = cv2.resize(image, (page_width, page_height), interpolation=cv2.INTER_AREA)

        results = model(source=image, conf=0.2, iou=0.8)[0] 
        
        detections = sv.Detections.from_ultralytics(results) 

        new_detections = []
        for item in detections:
            xyxy, mask, confidence, class_id, tracker_id, data = item
            new_detections.append(
                        {
                            'xyxy': xyxy,
                            'data': data,
                            'class_id': class_id
                        }
                    )
    
        sorted_detections_o = transpose_detection(image,new_detections)    
        sorted_detections_d = detect_filter(sorted_detections_o)
        sorted_detections_w = width_plus(sorted_detections_d)
        sorted_detections =  detect_filter_head(sorted_detections_w)
        words = page.get_text("words")
        
        
        for i,sorted_detection in enumerate(sorted_detections):
            
            if i < len(sorted_detections)-1:
                sub_class_name = sorted_detections[i+1]['data']['class_name']
            if i > 0  :
                 pre_class_name = sorted_detections[i-1]['data']['class_name']
                 p_x1,p_y1,p_x2,p_y2 = sorted_detections[i-1]['xyxy']

            x1, y1, x2, y2 = sorted_detection['xyxy']
            
            if sorted_detection['data']['class_name'] == 'Section-header' and temp != '' and pre_class_name !='Section-header' and not fitz.Rect(x1, y1, x2, y2).intersects(fitz.Rect(p_x1,p_y1,p_x2,p_y2)) and sub_class_name != 'Picture':
                
                
                
                docs[doc_id] = temp.lstrip()
                temp = ''
                doc_id += 1
                 
                 
            elif sorted_detection['data']['class_name'] == 'Page-footer':
                pass
                
            table_temp = ''
             
            for w in words: 
                if fitz.Rect(w[:4]).intersects(fitz.Rect(x1, y1, x2, y2)):
                    
                    if sorted_detection['data']['class_name'] == 'Table':
                        table_temp += w[4]
                        
                    elif  not fitz.Rect(w[:4]).intersects(fitz.Rect(p_x1,p_y1,p_x2,p_y2)):
                        temp += ' ' + w[4]
                        
            if sorted_detection['data']['class_name'] == 'Table':
                
                table_text = trans_table(markdown_tables,table_temp)
                
                pattern = r"\(단위: [^)]*\)"
                match_table = re.search(pattern, table_text)
                match_temp = re.search(pattern, temp) 
                
  
                if match_temp and match_table:
                      table_text = table_text.replace(match_table.group(),'')
                
                elif match_temp:
                      extracted_text = match_temp.group()
                    
                elif match_table:
                      extracted_text = match_table.group()
                else:
                      table_text = extracted_text + table_text
                
     
                temp += table_text
                
            temp += '\n'
            
    if temp != '':  
        
        if pre_class_name=='Text' and '집행절차' in temp and sorted_detection['data']['class_name'] == 'Picture':
            keyword = "사업 집행절차"
            index = temp.find(keyword)
            check_order = temp[index + len(keyword):]

            q = f"""
                            다음 정보를 바탕으로 요구사항에 답하세요:

                            {check_order}
                            
                            요구사항: 절차의 순서를 올바르게 정렬하고 ,로 구분해주세요. 

                            
                            주어진 요구사항만 답변하세요.

                            답변:
                            """

            answer = make_answer(q)
            temp = temp[:index+ len(keyword)] + ' ' + answer
            
        docs[doc_id] = temp.lstrip()
        if doc_id != 0:
            docs[doc_id-1] = docs[doc_id - 1].replace(docs[doc_id], '')
    
    for i in range(doc_id): 
        if i > len(docs):
            break
        
        low = len(docs[i])
        if low < 232:
            docs[i + 1] = docs[i] + docs[i+1]
            del docs[i]
        
      
          
       
    parser_path = './dacon/parser_docs'
    if not os.path.exists(parser_path):
        os.makedirs(parser_path,exist_ok=True)
    with open(f'./dacon/parser_docs/{folder_path.split("/")[-1]}.json', 'w', encoding='utf-8') as f:
        json.dump(docs, f, ensure_ascii=False, indent=4)
        

# pdf -> file name
pdf_path = './test_source'
files = [f.replace('.pdf', '') for f in os.listdir(pdf_path)]
for file_name in tqdm(files):
        detect_and_parser('./dacon/test_png/'+file_name, model_path='yolov10x_best.pt')




  0%|          | 0/9 [00:00<?, ?it/s]


0: 640x480 5 List-items, 2 Pictures, 1 Section-header, 6 Texts, 1 Title, 397.2ms
Speed: 0.0ms preprocess, 397.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)

0: 480x640 13 List-items, 1 Page-footer, 1 Page-header, 8 Section-headers, 1 Table, 3 Texts, 414.8ms
Speed: 0.0ms preprocess, 414.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 Footnotes, 20 List-items, 1 Page-footer, 1 Page-header, 3 Section-headers, 1 Table, 3 Texts, 395.9ms
Speed: 14.5ms preprocess, 395.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Footnote, 11 List-items, 1 Page-header, 7 Section-headers, 3 Tables, 11 Texts, 382.8ms
Speed: 2.0ms preprocess, 382.8ms inference, 15.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 18 List-items, 1 Page-footer, 6 Section-headers, 3 Tables, 9 Texts, 404.1ms
Speed: 0.0ms preprocess, 404.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 8 Footno

 11%|█         | 1/9 [00:11<01:29, 11.24s/it]


0: 640x480 1 List-item, 1 Picture, 2 Section-headers, 6 Texts, 1 Title, 401.8ms
Speed: 2.9ms preprocess, 401.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)

0: 480x640 1 Footnote, 19 List-items, 1 Page-header, 5 Section-headers, 4 Texts, 423.0ms
Speed: 10.3ms preprocess, 423.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Caption, 1 Footnote, 13 List-items, 2 Pictures, 3 Section-headers, 7 Texts, 469.6ms
Speed: 0.0ms preprocess, 469.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Footnote, 16 List-items, 1 Page-footer, 1 Page-header, 4 Pictures, 7 Section-headers, 9 Texts, 400.8ms
Speed: 0.1ms preprocess, 400.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 Footnotes, 21 List-items, 1 Page-footer, 4 Pictures, 6 Section-headers, 13 Texts, 386.1ms
Speed: 0.0ms preprocess, 386.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Capti

 22%|██▏       | 2/9 [00:25<01:29, 12.83s/it]

...

 78%|███████▊  | 7/9 [01:01<00:13,  6.69s/it]


0: 640x480 4 Captions, 1 Picture, 6 Tables, 551.1ms
Speed: 2.0ms preprocess, 551.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 Captions, 2 Section-headers, 2 Tables, 2 Texts, 674.0ms
Speed: 0.0ms preprocess, 674.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 List-items, 1 Section-header, 1 Table, 8 Texts, 475.4ms
Speed: 0.0ms preprocess, 475.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 4 Section-headers, 23 Texts, 585.0ms
Speed: 2.0ms preprocess, 585.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 3 List-items, 3 Section-headers, 2 Tables, 18 Texts, 535.7ms
Speed: 0.0ms preprocess, 535.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 Page-header, 2 Tables, 1 Text, 453.5ms
Speed: 5.1ms preprocess, 453.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 List-items, 1 Section-header, 

 89%|████████▉ | 8/9 [01:10<00:07,  7.64s/it]


0: 640x480 2 Captions, 3 List-items, 4 Section-headers, 6 Tables, 3 Texts, 1 Title, 434.8ms
Speed: 1.0ms preprocess, 434.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 19 List-items, 2 Section-headers, 9 Texts, 579.5ms
Speed: 1.0ms preprocess, 579.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 Caption, 1 Picture, 1 Table, 4 Texts, 412.2ms
Speed: 0.0ms preprocess, 412.2ms inference, 15.5ms postprocess per image at shape (1, 3, 640, 480)


100%|██████████| 9/9 [01:14<00:00,  8.25s/it]
